# Benji, Wendi, Yiwen

Dataset can be downloaded at:

https://www.kaggle.com/zynicide/wine-reviews

In [24]:
library(readr)
library(dplyr)
library(tidyr)
library(ggplot2)
library(gbm)
library(gam)
library(stringr)

Read in the data, drop duplicates, drop `description`, `winery`, and `taster_twitter_handle`.

In [2]:
read_csv("Data/wine-reviews/winemag-data-130k-v2.csv") %>% select(-X1) %>% unique %>% 
select(-description, -winery, -taster_twitter_handle) -> data

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  X1 = col_integer(),
  country = col_character(),
  description = col_character(),
  designation = col_character(),
  points = col_integer(),
  price = col_double(),
  province = col_character(),
  region_1 = col_character(),
  region_2 = col_character(),
  taster_name = col_character(),
  taster_twitter_handle = col_character(),
  title = col_character(),
  variety = col_character(),
  winery = col_character()
)


In this notebook, we will attempt to automate some of the data munging/imputation. 

We want to scale `points` by the taster, since the distribution of points is likely dependent on the taster. Therefore, we will attempt two different methods of unifying this distribution: scaling by standardization and scaling by percentile of score.

In [22]:
scale_taster <- function(points){
    # takes a vector of numbers, subtracts every element by the mean of the vector, and then
    # divides every element by the standard deviation of the vector
    
    return((points - mean(points, na.rm = TRUE)) / sd(points, na.rm = TRUE))
}

In [23]:
percentile_taster <- function(x){
    # takes a vector of numbers, ranks every element and divides by n, giving the percentile of each element
    trunc(rank(x))/length(x) * 100
}

In [19]:
data <- data %>% group_by(taster_name) %>% mutate("Scaled Points" = scale_taster(points))

In [20]:
data <- data %>% group_by(taster_name) %>% mutate("Percentile Points" = percentile_taster(points))

We want to reduce the number of provinces. If a given province only appears less than 1% of the time, we want to change that province to its country instead. However, we want to distinguish these miscellaneous provinces from the major ones, so we will call it `country_other`.

In [12]:
tab <- data %>% group_by(province) %>% summarize("Proportion" = n()/nrow(data))
tab <- tab[tab$Proportion > 0.01, ]

tabcountry <-  data %>% group_by(country) %>% summarize("Proportion" = n()/nrow(data))
tabcountry <- tabcountry[tabcountry$Proportion > 0.01, ]

In [13]:
data$country_other <- ifelse(data$country %in% tabcountry$country, 
                                paste0(data$country, "_other"), data$country)
data$location <- ifelse(data$province %in% tab$province, data$province,
                                     data$country_other)

Now that we have consolidated provinces under the `location` variable, we can drop `province` and `country_other` since we no longer have any need for them.

In [16]:
data <- data %>% select(-province, -country_other)

In [39]:
year <- str_extract_all(data$title, "[1-2][09][0-9]{2}")

In [40]:
data$year <- lapply(year, function(x){
    x = x %>% as.numeric
    if(!all(is.na(x))){
        #newx <- max(x[(x > 1900) & (x < 2018)])
        #return(newx)
        return(max(x))
    }
    else {
        return(NA) 
    }}) %>% unlist

In [41]:
summary(data$year)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   1904    2009    2011    2011    2013    2900    4285 

We want to impute `price` by the average price of the missing wine's variety, i.e. we will group by variety, take the mean, and then use that value to impute. However, we won't be able to impute the price of wines where the variety has only missing prices because then the mean wouldn't be defined. Thus, these cases will be removed.

In [4]:
impute_mean <- function(x) replace(x, is.na(x), mean(x, na.rm = TRUE))
    # impute_mean replaces missing values with the average value of a group

In [5]:
clean <- function(df){
    # clean removes the varieties that only have missing prices, and are thus unimputable by our rule,
    # and then it imputes the remaining missing prices using the average price of that wine's variety
    
    df %>% group_by(variety) %>% summarize("Average_Price" = mean(price, na.rm = T), 
                                           "Count" = n()) %>% 
    filter(is.na(Average_Price)) %>% select(variety) %>% unlist() -> drop_variety 
    
    df %>% filter(!(variety %in% drop_variety)) -> sample2
    
    return(sample2 %>% group_by(variety) %>% mutate(price = impute_mean(price)))
}

For our gradient boosted machine, it cannot handle variables with many levels. Thus, we must drop title and winery if we want our gbm function to operate correctly. We also want to drop region_1 and region_2 since this is part of our process for handling missing data.

K-Fold Cross-Validation:

In [ ]:
set.seed(2018)
k <- 10
sp <- split(c(1:nrow(sample)), c(1:k))
fold_error <- c()

for(i in 1:k){
    train <- sample[-sp[[k]], ]
    test <- sample[sp[[k]], ]
    
    cleanwine_train <- clean(train)
    cleanwine_test <- clean(test)
    
    cleanwine_train[which(is.na(cleanwine_train$taster_name)), "taster_name"] <- "Missing"
    cleanwine_test[which(is.na(cleanwine_test$taster_name)), "taster_name"] <- "Missing"

    cleanwine_train$country <- factor(cleanwine_train$country)
    cleanwine_train$province <- factor(cleanwine_train$province)
    cleanwine_train$taster_name <- factor(cleanwine_train$taster_name)
    cleanwine_train$title <- factor(cleanwine_train$title)
    cleanwine_train$variety <- factor(cleanwine_train$variety)
    cleanwine_train$winery <- factor(cleanwine_train$winery)
    
    cleanwine_test$country <- factor(cleanwine_test$country)
    cleanwine_test$province <- factor(cleanwine_test$province)
    cleanwine_test$taster_name <- factor(cleanwine_test$taster_name)
    cleanwine_test$title <- factor(cleanwine_test$title)
    cleanwine_test$variety <- factor(cleanwine_test$variety)
    cleanwine_test$winery <- factor(cleanwine_test$winery)

    cleanwine_train <- cleanwine_train %>% select(-title, -winery, -region_1, -region_2)
    cleanwine_test <- cleanwine_test %>% select(-title, -winery, -region_1, -region_2)

    
    gbmFit <- gbm(formula = price ~ ., data = cleanwine_train, 
                  n.trees = 1000, shrinkage = 0.01, interaction.depth = 2, cv.folds = 10, 
                  distribution = "laplace")

    best_iter <- gbm.perf(gbmFit, method = "cv")
    
    abs_error <- abs(cleanwine_test$price - predict(gbmFit, newdata = cleanwine_test, n.trees = best_iter))
    fold_error <- c(fold_error, mean(abs_error, na.rm = TRUE))
}

In [ ]:
error_df <- data.frame("KFold" = 1:k, "MAE" = fold_error)
fold <- which(error_df$MAE == min(error_df$MAE))
paste0("We will use fold #", fold, " to train our model.")

Now we will assess our model on the full data set.

In [ ]:
train <- sample[-sp[[8]], ]
    
cleanwine_train <- clean(train)
cleanwine_test <- clean(data %>% select(-description))
    
cleanwine_train[which(is.na(cleanwine_train$taster_name)), "taster_name"] <- "Missing"
cleanwine_test[which(is.na(cleanwine_test$taster_name)), "taster_name"] <- "Missing"

cleanwine_train$country <- factor(cleanwine_train$country)
cleanwine_train$province <- factor(cleanwine_train$province)
cleanwine_train$taster_name <- factor(cleanwine_train$taster_name)
cleanwine_train$title <- factor(cleanwine_train$title)
cleanwine_train$variety <- factor(cleanwine_train$variety)
cleanwine_train$winery <- factor(cleanwine_train$winery)
    
cleanwine_test$country <- factor(cleanwine_test$country)
cleanwine_test$province <- factor(cleanwine_test$province)
cleanwine_test$taster_name <- factor(cleanwine_test$taster_name)
cleanwine_test$title <- factor(cleanwine_test$title)
cleanwine_test$variety <- factor(cleanwine_test$variety)
cleanwine_test$winery <- factor(cleanwine_test$winery)

cleanwine_train <- cleanwine_train %>% select(-title, -winery, -region_1, -region_2)
cleanwine_test <- cleanwine_test %>% select(-title, -winery)
# cleanwine_test <- cleanwine_test %>% select(-title, -winery, -region_1, -region_2)
# note: our test data doesn't contain region_1 or region_2, so we must adjust our code accordingly...

    
gbmFit <- gbm(formula = price ~ ., data = cleanwine_train,
              n.trees = 1000, shrinkage = 0.01, interaction.depth = 2, cv.folds = 10,
              distribution = "laplace")

best_iter <- gbm.perf(gbmFit, method = "cv")
    
abs_error <- abs(cleanwine_test$price - predict(gbmFit, newdata = cleanwine_test, n.trees = best_iter))

In [ ]:
print(mean(abs_error)) # our performance on the test data